![](https://www.sparklebox.co.uk/wp-content/uploads/1-7068.jpg)


In [2]:
# Loading in the xmas.csv dataset
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


In [3]:
# Loading in the xmas_2.csv dataset
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


In [4]:
# Function 1: pluralize_gift()
def pluralize_gift(gift):
    """
    Returns plural of a noun

    Parameters
    ----------
    gift: str
      A noun

    Return
    ------
    str
      Plural version
    """

    irregular_plurals = {"goose": "geese"}

    if gift in irregular_plurals:
        return irregular_plurals[gift]
    elif gift.endswith("s"):
        return gift + "es"
    elif gift.endswith("y") and gift[-2] not in "aeiou":
        return gift[:-1] + "ies"
    else:
        return gift + "s"

# Designating a map of cardinal numbers
cardinal_numbers = {
    1: "a", 2: "two", 3: "three", 4: "four", 5: "five",
    6: "six", 7: "seven", 8: "eight", 9: "nine", 10: "ten",
    11: "eleven", 12: "twelve"
}

# Function2: make_phrase()
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Takes as input the necessary information, and returns a phrase

    Parameters
    ----------
    num: int
      Which day of Christmas
    num_word: str
      Cardinal number representation of the day
    item: str
      The gift item of the day
    verb: str
      Associated verb with gift
    adjective: str
      Optional descriptor of gift
    location: str
      Optional locator of gift

    Return
    ------
    str
      The constructed phrase for the specific day of Christmas
    """

    item = "" if pd.isna(item) else str(item)
    verb = "" if pd.isna(verb) else str(verb)
    adjective = "" if pd.isna(adjective) else str(adjective)
    location = "" if pd.isna(location) else str(location)

    # Using cardinal numbers for the gift item
    num_word_item = cardinal_numbers.get(num, str(num))

    # Determining whether to use 'a' or 'an' based on the word
    def use_a_or_an(word):
        vowels = "aeiou"
        return "an" if word[0].lower() in vowels else "a"

    # Pluralizing the gift item if num > 1
    item = pluralize_gift(item) if num > 1 else item

    # Constructing the phrase
    if num == 1:
        phrase = f"{use_a_or_an(item)} {item}"
    else:
        phrase = f"{num_word_item} {adjective + ' ' if adjective.lower() != 'na' and adjective != '' else ''}{item}"

    # Adding verb and location if they are not 'na' or empty
    if verb.lower() != 'na' and verb != '':
        phrase += f" {verb}"
    if location.lower() != 'na' and location != '':
        phrase += f" {location}"

    return phrase

# Creating the Full.Phrase column for xmas
xmas['Full.Phrase'] = xmas.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1
)

# Creating the Full.Phrase column for xmas2
xmas2['Full.Phrase'] = xmas2.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1
)

# Function 3: sing_day()
def sing_day(dataset, num, phrase_col):
    """
    Sings the constructed phrase for the specific day of Christmas

    Parameters
    ----------
    dataset: DataFrame
      Specify which dataframe that contains the song information
    num: int
      Specify the day for the lyrics to be generated
    phrase_col: str
      The name of the column that contains the full phrase

    Return
    ------
    str
      Generate the lyrics for the specific day of Christmas
    """

    num_word = dataset.loc[num-1, 'Day.in.Words']
    intro = f"On the {num_word} day of Christmas, my true love sent to me:"

    gifts = ",\n".join(dataset.loc[i, phrase_col] for i in range(num-1, -1, -1))

    # Handling 'and' before the last gift
    if num > 1:
        last_comma = gifts.rfind(",")
        gifts = gifts[:last_comma] + ", and" + gifts[last_comma+1:]

    return f"{intro}\n{gifts}."

# Function 4: sing_all_days()
def sing_all_days(day, dataframe):
    """
    Output the lyrics for the entire 12 Days of Christmas song
    Wrapper for the sing_day function

    Parameters
    ----------
    day: int64
        Which day's lyrics is generated
    dataframe: DataFrame
        Specifies the df for output

    Return
    ------
    str
        Outputs the lyrics for the specified days, including all previous days
    """

    return sing_day(dataframe, day, 'Full.Phrase')

# Function 5: clean_song()
def clean_song(song):
    """
    Remove any additional spaces between words, such that there should only be
    one whitespace character between words and no whitespace at the beginning or
    end of each phrase.

    Parameters
    ----------
    song : str
        The song text with lines separated by newline characters.

    Returns
    -------
    str
        The cleaned song text with one space between words and no leading/trailing whitespace on each line.
    """

    # Split the song into lines
    lines = song.split("\n")

    # Process each line to remove extra spaces using list comprehensions
    cleaned_lines = [" ".join(line.split()) for line in lines]

    # Join the cleaned lines back together with newline characters
    cleaned_song = "\n".join(cleaned_lines)

    return cleaned_song

In [5]:
# Singing the 12 days of Christmas for xmas.csv
song = "\n\n".join(map(lambda day: sing_all_days(day, xmas), range(1, 13)))

# Clean up the song
cleaned_song = clean_song(song)

print(cleaned_song)

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves, and
a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves, and
a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a 

In [6]:
# Singing the 12 days of Christmas for xmas.csv
song2 = "\n\n".join(map(lambda day: sing_all_days(day, xmas2), range(1, 13)))

# Clean up the song
cleaned_song2 = clean_song(song2)

print(cleaned_song2)

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points, and
an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points, and
an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.

On t